In [9]:
import torch 
import torch.nn as nn
import torchmetrics
from torch.utils.data import Dataset
import pandas as pd

In [ ]:
TRAIN_VAL_TEST_SPLIT = [0.9, 0.05, 0.05]

In [11]:

class SteelPlateDataset(Dataset):

    def __init__(self, dataset_path):
        super().__init__()
        self.path = dataset_path
        self.df = pd.read_csv(self.path)

        self.features = self.df.drop(["Class", *("Pastry Z_Scratch K_Scratch Stains Dirtiness Bumps".split(" "))] ,axis= 1).values.tolist() # V28 V29 V30 V31 V32 V33
        self.labels = self.df["Class"].to_list()

    def __getitem__(self, index):

        return torch.tensor(self.features[index]), torch.tensor(self.labels[index])
    def __len__(self):
        return len(self.labels)
        



In [12]:
torch.manual_seed(42)


dataset = SteelPlateDataset("data/norm_data.csv")

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, TRAIN_VAL_TEST_SPLIT)



In [13]:
len(dataset.features[0])

27

In [14]:
if torch.cuda.is_available():
    device_name = "cuda"
elif torch.backends.mps.is_available():
    device_name = "mps"
else:
    device_name = "cpu"

In [15]:
import torch.nn as nn 


BATCH_SIZE = 32
LR = 0.001
CLASSES = 2



EPOCHS = 40
LOSS_FUNCTION =nn.BCEWithLogitsLoss()

AUGMENT = True
SAVE_BEST_MODEL = True
IS_MULTICLASS = True if CLASSES > 2 else False
NUM_OF_WORKERS = 0

In [16]:
from torchmetrics.classification import Accuracy, Precision, F1Score
from torch.optim.lr_scheduler import ReduceLROnPlateau


class MyModel(nn.Module):
    def __init__(self, input_size,lr=0.001, loss_fn=nn.BCELoss(), num_classes=2):
        super().__init__()
        self.accuracy = Accuracy(task="binary", num_labels=num_classes)
        self.f1 = F1Score(task="binary", num_labels=num_classes, average='macro')
        self.loss_fn = loss_fn


        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
        self.to(device_name)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        self.scheduler = ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=10
        )

    def forward(self, x):
        return self.model(x)
        
        
    def evaluate(self, dataloader):
        self.eval()
        eval_loss = 0
        

        with torch.no_grad():
            for batch in dataloader:

                x = batch[0].to(device_name)
                y = batch[1].to(device_name)

                output = self.forward(x)
                output = torch.sigmoid(output).squeeze(1)
                y = y.float()

                
                    
                loss = self.loss_fn(output, y)
 
                self.accuracy(output, y)
              
          
                eval_loss += loss.item()

        self.train()
        return (eval_loss/len(dataloader), self.accuracy.compute(), self.f1.compute())
    
        
    def fit(self, train_dataloader, val_dataloader, epochs=10):
        self.train()
        best_val_loss = 9999

        train_loss_hist = []
        train_accuracy_hist = []
        train_f1_hist = []

        val_loss_hist = []
        val_accuracy_hist = []
        val_f1_hist = []

      
        for i in range(0,epochs):
           
            self.accuracy.reset()
            epoch_loss = 0
            for batch in train_dataloader:

                x = batch[0].to(device_name)
                y = batch[1].to(device_name)
              

                output = self.forward(x)
                output = torch.sigmoid(output).squeeze(1)
                y = y.float()
                
            
                
                loss = self.loss_fn(output, y)

               
                self.accuracy(output, y)
                self.f1(output, y)

                epoch_loss += loss.item()

                self.zero_grad()
                loss.backward()
                self.optimizer.step()


            epoch_loss/=len(train_dataloader)
           
            epoch_acc = self.accuracy.compute()
            epoch_f1 = self.f1.compute()

       
            train_accuracy_hist.append(epoch_acc.item())
            train_loss_hist.append(epoch_loss)
            train_f1_hist.append(epoch_f1.item())

            val_loss, val_acc, val_f1 = self.evaluate(val_dataloader)
            if best_val_loss > val_loss:
                best_val_loss = val_loss
                torch.save(self.state_dict(), "best-model-by-loss.pth")

            
            val_accuracy_hist.append(val_acc.item())
            val_loss_hist.append(val_loss)
            val_f1_hist.append(val_f1.item())

            self.scheduler.step(val_loss)
            #wandb.log({"epoch": i, "Train Loss": epoch_loss, "Train Acc":epoch_acc,"Train F1":epoch_f1, "Val Loss":val_loss, "Val Acc":val_acc,"Val F1":val_f1, "LR":self.optimizer.param_groups[0]['lr']})
            print(f"Epoch {i+1} Loss:{epoch_loss:.4f} Accuracy:{epoch_acc:.4f} F1:{epoch_f1:.4f}  Val Loss:{val_loss:.4f} Val Accuracy:{val_acc:.4f} Val F1:{val_f1:.4f} LR = {self.optimizer.param_groups[0]['lr']}")
        #wandb.finish()
        return (train_loss_hist, train_accuracy_hist,train_f1_hist), (val_loss_hist, val_accuracy_hist,val_f1_hist)


        

In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,  # Default shuffling for training
    num_workers=NUM_OF_WORKERS
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # No shuffling for validation
    num_workers=NUM_OF_WORKERS
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # No shuffling for test
    num_workers=NUM_OF_WORKERS
)

# Print dataset sizes
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 1747
Validation dataset size: 97
Test dataset size: 97


In [18]:
model = MyModel(input_size=27,num_classes=CLASSES, loss_fn=LOSS_FUNCTION, lr=LR)

In [19]:
model.fit(train_dataloader, val_dataloader, epochs=200)

Epoch 1 Loss:0.7019 Accuracy:0.6497 F1:0.0129  Val Loss:0.6932 Val Accuracy:0.6491 Val F1:0.0129 LR = 0.001
Epoch 2 Loss:0.6932 Accuracy:0.6520 F1:0.0065  Val Loss:0.6932 Val Accuracy:0.6513 Val F1:0.0065 LR = 0.001
Epoch 3 Loss:0.6931 Accuracy:0.6520 F1:0.0044  Val Loss:0.6932 Val Accuracy:0.6513 Val F1:0.0044 LR = 0.001
Epoch 4 Loss:0.6931 Accuracy:0.6520 F1:0.0033  Val Loss:0.6932 Val Accuracy:0.6513 Val F1:0.0033 LR = 0.001
Epoch 5 Loss:0.6931 Accuracy:0.6520 F1:0.0026  Val Loss:0.6931 Val Accuracy:0.6513 Val F1:0.0026 LR = 0.001
Epoch 6 Loss:0.6931 Accuracy:0.6520 F1:0.0022  Val Loss:0.6931 Val Accuracy:0.6513 Val F1:0.0022 LR = 0.001
Epoch 7 Loss:0.6931 Accuracy:0.6520 F1:0.0019  Val Loss:0.6931 Val Accuracy:0.6513 Val F1:0.0019 LR = 0.001
Epoch 8 Loss:0.6931 Accuracy:0.6520 F1:0.0016  Val Loss:0.6931 Val Accuracy:0.6513 Val F1:0.0016 LR = 0.001
Epoch 9 Loss:0.6931 Accuracy:0.6520 F1:0.0015  Val Loss:0.6931 Val Accuracy:0.6513 Val F1:0.0015 LR = 0.001
Epoch 10 Loss:0.6931 Accurac

(([0.7019132928414779,
   0.6931506568735296,
   0.6931497768922286,
   0.6931489250876687,
   0.6931480461900884,
   0.693147477236661,
   0.6931472832506353,
   0.6931471705436707,
   0.6931469711390409,
   0.6931121056730097,
   0.6875372333960099,
   0.6757726669311523,
   0.6731557640162381,
   0.6673652312972329,
   0.6674859339540655,
   0.6659338094971397,
   0.6660365928303111,
   0.6647123239257119,
   0.6630613760514693,
   0.6639198216524991,
   0.6634709520773454,
   0.6623709093440663,
   0.6630706906318664,
   0.6610744151202115,
   0.6625717434016141,
   0.6611752759326588,
   0.6594665657390247,
   0.6611161448738792,
   0.6625533114780079,
   0.6589552196589383,
   0.6601053162054582,
   0.6581683917479082,
   0.6591029925779863,
   0.6599001060832631,
   0.6589713974432512,
   0.6576126846400174,
   0.6568413918668573,
   0.6575597232038325,
   0.6564569722522389,
   0.6562481186606667,
   0.6564919883554632,
   0.6560956911607222,
   0.6553360028700395,
   0.6553703

In [20]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Example data (each pixel has an integer class label)
val_dataloader = DataLoader(
    test_dataset,
    batch_size=len(val_dataset),
    shuffle=False,  # Use sampler instead of shuffle
    num_workers=NUM_OF_WORKERS
)

features, ground_truth = next(iter(val_dataloader))

images = features.to(device_name)
model.eval()
predictions = model(images)
predictions = (torch.sigmoid(predictions) > 0.5).float()
y_true = ground_truth.numpy()
y_pred = predictions.detach().cpu().int().numpy()





In [21]:
cm = confusion_matrix(y_true, y_pred)
cm_percent = cm.astype('float') / cm.sum(axis=1, keepdims=True) * 100
disp = ConfusionMatrixDisplay(confusion_matrix=cm_percent,
                          display_labels=["Yes", "Not"])

disp.plot(cmap='Blues', values_format='.1f')



   
